# Toy OSC

Goal: Show possible issues with null-space and feedback control.

Two-dof prismatic setup kinda like example shown in [PS 2006]

```
 |->   |->  |
 q₀    q₁   x
```

with the following:

$$
\begin{align*}
    x &= q_0 + q_1 \\
    J &= \begin{bmatrix} 1 & 1 \end{bmatrix}
\end{align*}
$$

Below shows that the moment you drift off the null-space dynamics, you go unstable unless
you negate your position gains?

## Refs

* [PS 2006] Peters, Jan, and Stefan Schaal. “Learning Operational Space Control,” 2006. https://doi.org/10.15607/RSS.2006.II.033.

In [ ]:
import numpy as np
import scipy

In [ ]:
J = np.array([[1, 1]])

In [ ]:
# These null-spaces are effectively unused, but just show some basic properties.
Jpinv = np.linalg.pinv(J)
N = np.eye(2) - Jpinv @ J
print(Jpinv)
print(N)
# Compute some other null spaces - they're just reductions of the above
# (I think?).
N2 = scipy.linalg.null_space(J)
U, s, V_T = np.linalg.svd(J)
N3 = V_T[:, -1:]
print(N2)
print(N3)

In [ ]:
M = np.eye(2)
Minv = np.linalg.inv(M)
Mtinv = J @ J.T  # 2
Mt = np.linalg.inv(Mtinv)
# Same as nominal null-space.
Jbar = Minv @ J.T @ Mt  # Jpinv, dynamic
print(Jbar)
Nt = np.eye(2) - Jbar @ J
Nt_T = Nt.T
print(Nt)

In [ ]:
def naive_euler(q, v, vd_func, t0, tf, dt):
    t = t0
    ts = []
    qs = []
    q = q.copy()
    v = v.copy()
    while t <= tf:
        qs.append(q)
        ts.append(
        vd = vd_func(q, v)
        q = q + v * dt
        v = v + vd * dt
        qs.append(q)
    ts, qs = map(np.asarray, (ts, qs))
    return ts, qs

In [ ]:
q0 = np.array([0.0, 2.0])

def calc_vd(q, v):
    x = q[0] + q[1]
    x_des = 3.0
    e_t = x - x_des
    ed_t = J @ v
    kp_t = 1.0
    kd_t = 1.0
    edd_t_c = -kp_t * e_t - kd_t * ed_t

    e_p = q - q0
    ed_p = v
    kp_p = 1.0
    kd_p = 1.0
    edd_p_c = -kp_p * e_p - kd_p * ed_p

    tau = J.T @ Mt @ edd_t_c + Nt_T @ M @ edd_p_c
    vd = Minv @ tau
    return vd

# q = np.array([0.1, 2.1])
q = np.array([0.1, 1.9])
v = np.array([0.0, 0.0])
vd = calc_vd(q, v)
print(vd)

# WARNING: Instability in null-space!